## Purpose
Reusable code, utils etc. 

In [5]:
import pandas as pd
import numpy as np
import random

In [1]:
"""
Check if the current code is running in a notebook
From https://stackoverflow.com/questions/15411967/how-can-i-check-if-code-is-executed-in-the-ipython-notebook
"""
def isnotebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

## Tabular Data

In [16]:
import pandas as pd

def get_categorical_columns(df, ignore_cols=[]):
    return [o for o in df.select_dtypes(include=['category','object']).columns if o not in ignore_cols]

def get_numeric_columns(df, ignore_cols=[]):
    return [o for o in df.select_dtypes(exclude=['category','object']).columns if o not in ignore_cols]


### Imputation

In [34]:
df = pd.DataFrame({
    'num': [1,2,3,4,5,6,7,8,9,None],
    's': ['a','a','b','a','a','a','b','a',np.nan,'b'],
})

cat_cols = get_categorical_columns()
cont_cols = get_numeric_columns(df)

TypeError: get_categorical_columns() missing 1 required positional argument: 'df'

In [33]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

transforms = []

# impute
transforms.append(('imp_cat', SimpleImputer(strategy='constant', fill_value='-999'), cat_cols))
transforms.append(('imp_cont' + col, SimpleImputer(strategy='mean'), cont_cols))
column_trans = ColumnTransformer(transforms, remainder='passthrough')

column_trans.fit_transform(df_train)

NameError: name 'cat_cols' is not defined

### Outliers
remove outliers using percentiles

In [14]:
x = pd.Series(list(np.random.rand(50)) + [2.0])
upper_bound, lower_bound = np.percentile(x, [1,99])
x_clipped = np.clip(x, upper_bound, lower_bound)

### Scalers
https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html

Which scaler to choose? - https://docs.google.com/spreadsheets/d/1woVi7wq13628HJ-tN6ApaRGVZ85OdmHsDBKLAf5ylaQ/edit#gid=0